# Wave Equation in Cartesian Coordinates: Symbolic Expressions and `c_codegen()

## Authors: Zach Etienne & Thiago Assumpcao

## This module generates the C code for the wave equation in Cartesian coordinates and sets up either a monochromatic plane wave or a spherical Gaussian [initial data](https://en.wikipedia.org/wiki/Initial_value_problem) configuration. The module emphasizes the use of the Method of Lines, transforming the partial differential equation (PDE) problem into a more manageable ordinary differential equation (ODE) problem.

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy modules, as documented below ([right-hand-side expressions](#Step-3.a:-Validate-SymPy-expressions-against-WaveEquation_RHSs-NRPy-module); [initial data expressions](#Step-5:-Code-Validation-against-WaveEquation_Solutions_InitialData-NRPy-module)). In addition, all expressions have been validated against a trusted code (the [original SENR/NRPy code](https://bitbucket.org/zach_etienne/nrpy)).

### NRPy Source Code for this module:
* [ScalarWave/ScalarWave_RHSs.py](../edit/ScalarWave/ScalarWave_RHSs.py)
* [ScalarWave/InitialData.py](../edit/ScalarWave/InitialData.py)

## Introduction:
### Problem Statement

We wish to numerically solve the wave equation as an [initial value problem](https://en.wikipedia.org/wiki/Initial_value_problem) in Cartesian coordinates:
$$\partial_t^2 u = c^2 \nabla^2 u \text{,}$$
where $u$ (the amplitude of the wave) is a function of time and space: $u = u(t,x,y,...)$ (spatial dimension as yet unspecified) and $c$ is the wave speed, subject to some initial condition

$$u(0,x,y,...) = f(x,y,...)$$

and suitable spatial boundary conditions.

As described in the next section, we will find it quite useful to define
$$v(t,x,y,...) = \partial_t u(t,x,y,...).$$

In this way, the second-order PDE is reduced to a set of two coupled first-order PDEs

\begin{align}
\partial_t u &= v \\
\partial_t v &= c^2 \nabla^2 u.
\end{align}

We will use NRPy to generate efficient C code capable of producing both initial data $u(0,x,y,...) = f(x,y,...)$; $v(0,x,y,...)=g(x,y,...)$, as well as finite-difference expressions for the right-hand sides of the above expressions. These expressions are needed within the *Method of Lines* to "integrate" the solution forward in time.

### The Method of Lines

Once we have initial data, we "evolve it forward in time" using the [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html). In short, the Method of Lines enables us to handle
1. the **spatial derivatives** of an initial value problem PDE using **standard finite difference approaches**, and
2. the **temporal derivatives** of an initial value problem PDE using **standard strategies for solving ordinary differential equations (ODEs)**, so long as the initial value problem PDE can be written in the form
$$\partial_t \vec{f} = \mathbf{M}\ \vec{f},$$
where $\mathbf{M}$ is an $N\times N$ matrix filled with differential operators that act on the $N$-element column vector $\vec{f}$. $\mathbf{M}$ may not contain $t$ or time derivatives explicitly; only *spatial* partial derivatives are allowed to appear inside $\mathbf{M}$. The wave equation as written in the [previous module](Tutorial-ScalarWave.ipynb),
\begin{equation}
\partial_t
\begin{bmatrix}
u \\
v
\end{bmatrix}=
\begin{bmatrix}
0 & 1 \\
c^2 \nabla^2  & 0
\end{bmatrix}
\begin{bmatrix}
u \\
v
\end{bmatrix},
\end{equation}
satisfies this requirement.

Thus we can treat the spatial derivatives $\nabla^2 u$ of the wave equation using **standard finite difference approaches**, and the temporal derivatives $\partial_t u$ and $\partial_t v$ using **standard approaches for solving ODEs**. In [the next module](Tutorial-Start_to_Finish-ScalarWave.ipynb), we will apply the highly robust [explicit Runge-Kutta fourth-order scheme](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) (RK4), used widely for numerically solving ODEs, to "march" (integrate) the solution vector $\vec{f}$ forward in time from its initial value ("initial data").

### Basic Algorithm

The basic algorithm for solving the wave equation [initial value problem](https://en.wikipedia.org/wiki/Initial_value_problem), based on the Method of Lines (see section above), is outlined below, with NRPy-based components highlighted in <font color='green'>green</font>. We will review how NRPy generates these core components in this module.

1. Allocate memory for gridfunctions, including temporary storage for the RK4 time integration.
1. <font color='green'>Set gridfunction values to initial data.</font>
1. Evolve the system forward in time using RK4 time integration. At each RK4 substep, do the following:
    1. <font color='green'>Evaluate wave RHS expressions.</font>
    1. Apply boundary conditions.

**We refer to the right-hand side of the equation $\partial_t \vec{f} = \mathbf{M}\ \vec{f}$ as the RHS. In this case, we refer to the $\mathbf{M}\ \vec{f}$ as the "wave RHSs".** In the following sections we will

1. use NRPy to cast the wave RHS expressions in finite difference form into highly efficient C code,
    1. first in one spatial dimension with fourth-order finite differences,
    1. and then in three spatial dimensions with tenth-order finite differences; we will
1. use NRPy to generate monochromatic plane-wave initial data for the wave equation, where the wave propagates in an arbitrary direction.

As for the $\nabla^2 u$ term, spatial derivatives are handled in NRPy via [finite differencing](https://en.wikipedia.org/wiki/Finite_difference).

We will sample the solution $\{u,v\}$ at discrete, uniformly sampled points in space and time. For simplicity, let's assume that we consider the wave equation in one spatial dimension. Then the solution at any sampled point in space and time is given by
$$u^n_i = u(t_n,x_i) = u(t_0 + n \Delta t, x_0 + i \Delta x),$$
where $\Delta t$ and $\Delta x$ represent the temporal and spatial resolution, respectively. $v^n_i$ is sampled at the same points in space and time.

# Table of Contents

1. [Step 1](#Step-1:-Initialize-core-NRPy-modules-and-define-CodeParameters): Initialize core NRPy modules and define CodeParameters
1. [Step 2](#Step-2:-Scalar-Wave-RHSs-in-One-Spatial-Dimension): Wave RHSs in One Spatial Dimension, Fourth-Order Finite Differencing
    1. [Step 2.a](#Step-2.a:-C-code-output-example:-Scalar-wave-RHSs-with-fourth-order-finite-difference-stencils): C code output example: Scalar wave RHSs with fourth-order finite difference stencils
1. [Step 3](#Step-3:-Scalar-Wave-RHSs-in-Three-Spatial-Dimensions): Wave RHSs in Three Spatial Dimensions
    1. [Step 3.a](#Step-3.a:-Validate-SymPy-expressions-against-WaveEquation_RHSs-NRPy-module): Validate SymPy expressions against WaveEquation_RHSs NRPy module
    1. [Step 3.b](#Step-3.b:-C-code-output-example:-Scalar-wave-RHSs-with-10th-order-finite-difference-stencils-and-SIMD-enabled): C code output example: Scalar wave RHSs with 10th order finite difference stencils and SIMD enabled
1. [Step 4](#Step-4:-Setting-up-Initial-Data-for-the-Scalar-Wave-Equation): Setting up Initial Data for the Wave Equation
    1. [Step 4.a](#Step-4.a:-The-Monochromatic-Plane-Wave-Solution): The Monochromatic Plane-Wave Solution
    1. [Step 4.b](#Step-4.b:-The-Spherical-Gaussian-Solution): The Spherical Gaussian Solution (*Courtesy Thiago Assumpcao*)
1. [Step 5](#Step-5:-Code-Validation-against-WaveEquation_Solutions_InitialData-NRPy-module): Code Validation against WaveEquation_Solutions_InitialData NRPy module
1. [Step 6](#Step-6:-Output-this-notebook-to-LaTeX-formatted-PDF-file): Output this notebook to $\LaTeX$-formatted PDF file

# Step 1: Initialize core NRPy modules and define CodeParameters
### \[Back to [top](#Table-of-Contents)\]

Let's start by importing the needed modules from NRPy and defining the CodeParameters that will be used throughout this tutorial.

In [1]:
# Step 1a: Import needed NRPy core modules:
import nrpy.params as par      # NRPy: Parameter interface
import nrpy.indexedexp as ixp  # NRPy: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import nrpy.grid as gri        # NRPy: Functions having to do with numerical grids
import nrpy.c_codegen as ccg   # NRPy: Core C code output module
import sympy as sp             # SymPy: The Python computer algebra package upon which NRPy depends
import sys                     # For sys.exit in validation

# Set infrastructure to BHaH (needed for BHaHGridFunction access)
par.set_parval_from_str("Infrastructure", "BHaH")

# Step 1b: Define all CodeParameters used in the notebook.
thismodule = "ScalarWave"
wavespeed = par.register_CodeParameter("REAL", thismodule, "wavespeed", 1.0)
time = par.register_CodeParameter("REAL", thismodule, "time", 0.0)
kk0 = par.register_CodeParameter("REAL", thismodule, "kk0", 1.0)
kk1 = par.register_CodeParameter("REAL", thismodule, "kk1", 1.0)
kk2 = par.register_CodeParameter("REAL", thismodule, "kk2", 1.0)
sigma = par.register_CodeParameter("REAL", thismodule, "sigma", 2.0)

# Step 2: Wave RHSs in One Spatial Dimension
### \[Back to [top](#Table-of-Contents)\]

To minimize complication, we will first restrict ourselves to solving the wave equation in one spatial dimension, so
\begin{align}
\partial_t u &= v \\
\partial_t v &= c^2 \nabla^2 u \\
 &= c^2 \partial_x^2 u.
\end{align}

We will construct SymPy expressions for the right-hand sides of $u$ and $v$ using [NRPy finite-difference notation](Tutorial-Finite_Difference_Derivatives.ipynb) to represent the derivative, so that finite-difference C-code kernels can be easily constructed.

Extension of this operator to higher spatial dimensions when using NRPy is straightforward, as we will see below.

In [2]:
# Step 2.1: Set the spatial dimension parameter to 1.
dimension = 1

# Step 2.2: Register gridfunctions that are needed as input
#           to the wave RHS expressions.
uu, vv = gri.register_gridfunctions(["uu", "vv"], group="EVOL")

# Step 2.3: Declare the rank-2 indexed expression \partial_{ij} u,
#           which is symmetric about interchange of indices i and j.
#           Derivative variables like these must have an underscore
#           in them, so the finite difference module can parse the
#           variable name properly.
uu_dDD = ixp.declarerank2("uu_dDD", symmetry="sym01")

# Step 2.4: Define right-hand sides for the evolution.
uu_rhs = vv
vv_rhs = 0
for i in range(dimension):
    vv_rhs += wavespeed*wavespeed*uu_dDD[i][i]

vv_rhs = sp.simplify(vv_rhs)

## Step 2.a: C code output example: Scalar wave RHSs with fourth-order finite difference stencils
### \[Back to [top](#Table-of-Contents)\]

As was discussed in [the finite difference section of the tutorial](Tutorial-Finite_Difference_Derivatives.ipynb), NRPy approximates derivatives using [finite-difference methods](https://en.wikipedia.org/wiki/Finite_difference). The second spatial derivative $\partial_x^2$, accurate to fourth order in the uniform grid spacing $\Delta x$ (from fitting the unique 4th-degree polynomial to 5 sample points of $u$), is given by
\begin{equation}
\left[\partial_x^2 u(t,x)\right]_j = \frac{1}{(\Delta x)^2}
\left(
-\frac{1}{12} \left(u_{j+2} + u_{j-2}\right)
+ \frac{4}{3}  \left(u_{j+1} + u_{j-1}\right)
- \frac{5}{2} u_j \right)
+ \mathcal{O}\left((\Delta x)^4\right).
\end{equation}

In [3]:
# Step 2.a.1: Set the finite differencing order to 4.
par.set_parval_from_str("fd_order", 4)

# Step 2.a.2: Generate C code for scalarwave evolution equations,
#             print output to the screen (standard out, or stdout).
output = ccg.c_codegen(
    [uu_rhs, vv_rhs],
    [
        gri.BHaHGridFunction.access_gf("uu", gf_array_name="rhs_gfs"),
        gri.BHaHGridFunction.access_gf("vv", gf_array_name="rhs_gfs"),
    ],
    enable_fd_codegen=True,
    enable_simd=False,
)
print(output)

/*
 * NRPy-Generated GF Access/FD Code, Step 1 of 2:
 * Read gridfunction(s) from main memory and compute FD stencils as needed.
 */
const REAL uu_i0m2 = in_gfs[IDX4(UUGF, i0-2, i1, i2)];
const REAL uu_i0m1 = in_gfs[IDX4(UUGF, i0-1, i1, i2)];
const REAL uu = in_gfs[IDX4(UUGF, i0, i1, i2)];
const REAL uu_i0p1 = in_gfs[IDX4(UUGF, i0+1, i1, i2)];
const REAL uu_i0p2 = in_gfs[IDX4(UUGF, i0+2, i1, i2)];
const REAL vv = in_gfs[IDX4(VVGF, i0, i1, i2)];
static const REAL FDPart1_Rational_5_2 = 5.0/2.0;
static const REAL FDPart1_Rational_1_12 = 1.0/12.0;
static const REAL FDPart1_Rational_4_3 = 4.0/3.0;
const REAL uu_dDD00 = ((invdxx0)*(invdxx0))*(FDPart1_Rational_1_12*(-uu_i0m2 - uu_i0p2) + FDPart1_Rational_4_3*(uu_i0m1 + uu_i0p1) - FDPart1_Rational_5_2*uu);

/*
 * NRPy-Generated GF Access/FD Code, Step 2 of 2:
 * Evaluate SymPy expressions and write to main memory.
 */
rhs_gfs[IDX4(UUGF, i0, i1, i2)] = vv;
rhs_gfs[IDX4(VVGF, i0, i1, i2)] = uu_dDD00*((wavespeed)*(wavespeed));



# Step 3: Wave RHSs in Three Spatial Dimensions
### \[Back to [top](#Table-of-Contents)\]

Let's next repeat the same process, only this time for the wave equation in **3 spatial dimensions** (3D).

In [4]:
# Step 3.1: Set the spatial dimension to 3.
dimension = 3

# Step 3.2: Reset gridfunctions registered in 1D case above,
#           to avoid NRPy throwing an error about double-
#           registering gridfunctions, which is not allowed.
gri.glb_gridfcs_dict.clear()

# Step 3.3: Register gridfunctions that are needed as input
#           to the wave RHS expressions.
uu, vv = gri.register_gridfunctions(["uu", "vv"], group="EVOL")

# Step 3.4: Declare the rank-2 indexed expression \partial_{ij} u,
#           which is symmetric about interchange of indices i and j.
uu_dDD = ixp.declarerank2("uu_dDD", symmetry="sym01")

# Step 3.5: Define right-hand sides for the evolution.
uu_rhs = vv
vv_rhs = 0
for i in range(dimension):
    vv_rhs += wavespeed*wavespeed*uu_dDD[i][i]

vv_rhs = sp.simplify(vv_rhs)

## Step 3.a: Validate SymPy expressions against WaveEquation_RHSs NRPy module
### \[Back to [top](#Table-of-Contents)\]

Here, as a code validation check, we verify agreement in the SymPy expressions for the RHSs of the wave equation in three spatial dimensions (that is, `uu_rhs` and `vv_rhs`) between

1. this tutorial and
2. the [NRPy WaveEquation_RHSs](../edit/ScalarWave/ScalarWave_RHSs.py) module.

In [5]:
# Step 3.a.1: Reset the gridfunction dictionary to avoid conflicts.
gri.glb_gridfcs_dict.clear()

# Step 3.a.2: Import the WaveEquation_RHSs module and instantiate it.
from nrpy.equations.wave_equation.WaveEquation_RHSs import WaveEquation_RHSs
swrhs = WaveEquation_RHSs()

# Step 3.a.3: Perform consistency check.
print("Consistency check between ScalarWave tutorial and NRPy module:")
print("uu_rhs - swrhs.uu_rhs = " + str(sp.simplify(uu_rhs - swrhs.uu_rhs)) + "\t\t (should be zero)")
print("vv_rhs - swrhs.vv_rhs = " + str(sp.simplify(vv_rhs - swrhs.vv_rhs)) + "\t\t (should be zero)")

Consistency check between ScalarWave tutorial and NRPy module:
uu_rhs - swrhs.uu_rhs = 0		 (should be zero)
vv_rhs - swrhs.vv_rhs = 0		 (should be zero)


## Step 3.b: C code output example: Scalar wave RHSs with 10th order finite difference stencils and SIMD enabled
### \[Back to [top](#Table-of-Contents)\]

Next, we will output the above expressions as C code, using the [NRPy finite differencing C code kernel generation infrastructure](Tutorial-Finite_Difference_Derivatives.ipynb). This code will represent spatial derivatives as 10th-order finite differences and output the C code with [SIMD](https://en.wikipedia.org/wiki/SIMD) enabled. ([Common-subexpression elimination](https://en.wikipedia.org/wiki/Common_subexpression_elimination) is enabled by default.)

In [6]:
# Step 3.b.1: Set the finite differencing order to 10.
par.set_parval_from_str("fd_order", 10)

# Step 3.b.2: Generate C code for scalarwave evolution equations with SIMD.
output = ccg.c_codegen(
    [uu_rhs, vv_rhs],
    [
        gri.BHaHGridFunction.access_gf("uu", gf_array_name="rhs_gfs"),
        gri.BHaHGridFunction.access_gf("vv", gf_array_name="rhs_gfs"),
    ],
    enable_fd_codegen=True,
    enable_simd=True,
)
print(output)

/*
 * NRPy-Generated GF Access/FD Code, Step 1 of 2:
 * Read gridfunction(s) from main memory and compute FD stencils as needed.
 */
const REAL_SIMD_ARRAY uu_i2m5 = ReadSIMD(&in_gfs[IDX4(UUGF, i0, i1, i2-5)]);
const REAL_SIMD_ARRAY uu_i2m4 = ReadSIMD(&in_gfs[IDX4(UUGF, i0, i1, i2-4)]);
const REAL_SIMD_ARRAY uu_i2m3 = ReadSIMD(&in_gfs[IDX4(UUGF, i0, i1, i2-3)]);
const REAL_SIMD_ARRAY uu_i2m2 = ReadSIMD(&in_gfs[IDX4(UUGF, i0, i1, i2-2)]);
const REAL_SIMD_ARRAY uu_i2m1 = ReadSIMD(&in_gfs[IDX4(UUGF, i0, i1, i2-1)]);
const REAL_SIMD_ARRAY uu_i1m5 = ReadSIMD(&in_gfs[IDX4(UUGF, i0, i1-5, i2)]);
const REAL_SIMD_ARRAY uu_i1m4 = ReadSIMD(&in_gfs[IDX4(UUGF, i0, i1-4, i2)]);
const REAL_SIMD_ARRAY uu_i1m3 = ReadSIMD(&in_gfs[IDX4(UUGF, i0, i1-3, i2)]);
const REAL_SIMD_ARRAY uu_i1m2 = ReadSIMD(&in_gfs[IDX4(UUGF, i0, i1-2, i2)]);
const REAL_SIMD_ARRAY uu_i1m1 = ReadSIMD(&in_gfs[IDX4(UUGF, i0, i1-1, i2)]);
const REAL_SIMD_ARRAY uu_i0m5 = ReadSIMD(&in_gfs[IDX4(UUGF, i0-5, i1, i2)]);
const REAL_SIMD_ARRA

# Step 4: Setting up Initial Data for the Wave Equation
### \[Back to [top](#Table-of-Contents)\]

## Step 4.a: The Monochromatic Plane-Wave Solution
### \[Back to [top](#Table-of-Contents)\]

The solution to the wave equation for a monochromatic (single-wavelength) wave traveling in the $\hat{k}$ direction is
$$u(\vec{x},t) = f(\hat{k}\cdot\vec{x} - c t),$$
where $\hat{k}$ is a unit vector. We choose $f(\hat{k}\cdot\vec{x} - c t)$ to take the form
$$
f(\hat{k}\cdot\vec{x} - c t) = \sin\left(\hat{k}\cdot\vec{x} - c t\right) + 2,
$$
where we add the $+2$ to ensure that the exact solution never crosses through zero. In places where the exact solution passes through zero, the relative error (that is, the most common error measure used to check that the numerical solution converges to the exact solution) is undefined. Also, $f(\hat{k}\cdot\vec{x} - c t)$ plus a constant is still a solution to the wave equation.

In [7]:
# Step 4.a.1: Retrieve Cartesian coordinates (already set by dimension=3).
xCart = ixp.declarerank1("xCart", dimension=dimension)

# Step 4.a.2: Build k vector from the registered CodeParameters.
kk = [kk0, kk1, kk2]

# Step 4.a.3: Normalize the k vector.
kk_norm = sp.sqrt(kk[0]**2 + kk[1]**2 + kk[2]**2)

# Step 4.a.4: Compute k·x.
dot_product = sp.sympify(0)
for i in range(dimension):
    dot_product += xCart[i] * kk[i]
dot_product /= kk_norm

# Step 4.a.5: Set initial data for u and v.
uu_ID_PlaneWave = sp.sin(dot_product - wavespeed * time) + 2
vv_ID_PlaneWave = sp.diff(uu_ID_PlaneWave, time)

Next we verify that $f(\hat{k}\cdot\vec{x} - c t)$ satisfies the wave equation by computing
$$\left(c^2 \nabla^2 - \partial_t^2 \right)\ f\left(\hat{k}\cdot\vec{x} - c t\right),$$
and confirming that the result is exactly zero.

In [8]:
sp.simplify(wavespeed**2 * (sp.diff(uu_ID_PlaneWave, xCart[0], 2) +
                            sp.diff(uu_ID_PlaneWave, xCart[1], 2) +
                            sp.diff(uu_ID_PlaneWave, xCart[2], 2))
            - sp.diff(uu_ID_PlaneWave, time, 2))

0

## Step 4.b: The Spherical Gaussian Solution
### \[Back to [top](#Table-of-Contents)\]

Here we implement the spherical Gaussian solution, which consists of ingoing and outgoing wavefronts:
\begin{align}
u(r,t) &= u_{\rm out}(r,t) + u_{\rm in}(r,t) + 2,\ \ \text{where}\\
u_{\rm out}(r,t) &=\frac{r-ct}{r} \exp\left[\frac{-(r-ct)^2}{2 \sigma^2}\right] \\
u_{\rm in}(r,t) &=\frac{r+ct}{r} \exp\left[\frac{-(r+ct)^2}{2 \sigma^2}\right] \\
\end{align}
where $c$ is the wave speed, and $\sigma$ is the width of the Gaussian (that is, the "standard deviation").

In [9]:
# Step 4.b.1: Compute r = sqrt(x^2 + y^2 + z^2).
r = sp.sqrt(xCart[0]**2 + xCart[1]**2 + xCart[2]**2)

# Step 4.b.2: Set initial data for u and v using the spherical Gaussian formulas.
uu_ID_SphericalGaussianOUT = ((r - wavespeed * time) / r) * sp.exp(-(r - wavespeed * time)**2 / (2 * sigma**2))
uu_ID_SphericalGaussianIN  = ((r + wavespeed * time) / r) * sp.exp(-(r + wavespeed * time)**2 / (2 * sigma**2))
uu_ID_SphericalGaussian = uu_ID_SphericalGaussianOUT + uu_ID_SphericalGaussianIN + 2
vv_ID_SphericalGaussian = sp.diff(uu_ID_SphericalGaussian, time)

Since the wave equation is linear, both the ingoing and outgoing waves must satisfy the wave equation, which implies that their sum also satisfies the wave equation.

Next we verify that $u(r,t)$ satisfies the wave equation by computing
$$\left(c^2 \nabla^2 - \partial_t^2 \right)\left\{u_{\rm out}(r,t)\right\},$$

and

$$\left(c^2 \nabla^2 - \partial_t^2 \right)\left\{u_{\rm in}(r,t)\right\},$$

and confirming that each expression is separately zero. We do this because SymPy has difficulty simplifying the combined expression.

In [10]:
print(sp.simplify(wavespeed**2 * (sp.diff(uu_ID_SphericalGaussianOUT, xCart[0], 2) +
                                  sp.diff(uu_ID_SphericalGaussianOUT, xCart[1], 2) +
                                  sp.diff(uu_ID_SphericalGaussianOUT, xCart[2], 2))
                  - sp.diff(uu_ID_SphericalGaussianOUT, time, 2)))

print(sp.simplify(wavespeed**2 * (sp.diff(uu_ID_SphericalGaussianIN, xCart[0], 2) +
                                  sp.diff(uu_ID_SphericalGaussianIN, xCart[1], 2) +
                                  sp.diff(uu_ID_SphericalGaussianIN, xCart[2], 2))
                  - sp.diff(uu_ID_SphericalGaussianIN, time, 2)))

0
0


# Step 5: Code Validation against WaveEquation_Solutions_InitialData NRPy module
### \[Back to [top](#Table-of-Contents)\]

As a code validation check, we will verify agreement in the SymPy expressions for plane-wave and spherical-Gaussian initial data for the wave equation between
1. this tutorial and
2. the NRPy `nrpy.equations.wave_equation.WaveEquation_Solutions_InitialData` module.

In [11]:
# Step 5.1: Import the WaveEquation_Solutions_InitialData module and create instances.
from nrpy.equations.wave_equation.WaveEquation_Solutions_InitialData import WaveEquation_solution_Cartesian

# Plane wave
swid_plane = WaveEquation_solution_Cartesian(WaveType="PlaneWave", default_k0=1.0, default_k1=1.0, default_k2=1.0)

print("Consistency check between ScalarWave tutorial and NRPy module: PlaneWave Case")
if sp.simplify(uu_ID_PlaneWave - swid_plane.uu_exactsoln) != 0:
    print("TEST FAILED: uu_ID_PlaneWave - swid_plane.uu_exactsoln = " + str(sp.simplify(uu_ID_PlaneWave - swid_plane.uu_exactsoln)) + "\t\t (should be zero)")
    sys.exit(1)
if sp.simplify(vv_ID_PlaneWave - swid_plane.vv_exactsoln) != 0:
    print("TEST FAILED: vv_ID_PlaneWave - swid_plane.vv_exactsoln = " + str(sp.simplify(vv_ID_PlaneWave - swid_plane.vv_exactsoln)) + "\t\t (should be zero)")
    sys.exit(1)
print("TESTS PASSED!")

# Spherical Gaussian
swid_sph = WaveEquation_solution_Cartesian(WaveType="SphericalGaussian", default_sigma=2.0)

print("Consistency check between ScalarWave tutorial and NRPy module: SphericalGaussian Case")
if sp.simplify(uu_ID_SphericalGaussian - swid_sph.uu_exactsoln) != 0:
    print("TEST FAILED: uu_ID_SphericalGaussian - swid_sph.uu_exactsoln = " + str(sp.simplify(uu_ID_SphericalGaussian - swid_sph.uu_exactsoln)) + "\t\t (should be zero)")
    sys.exit(1)
if sp.simplify(vv_ID_SphericalGaussian - swid_sph.vv_exactsoln) != 0:
    print("TEST FAILED: vv_ID_SphericalGaussian - swid_sph.vv_exactsoln = " + str(sp.simplify(vv_ID_SphericalGaussian - swid_sph.vv_exactsoln)) + "\t\t (should be zero)")
    sys.exit(1)
print("TESTS PASSED!")

Consistency check between ScalarWave tutorial and NRPy module: PlaneWave Case
TESTS PASSED!
Consistency check between ScalarWave tutorial and NRPy module: SphericalGaussian Case
TESTS PASSED!
